<font size="6"><b>Lecture 06: GEE Image Manipulation (exercises)</b></font>

# Initialize

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='gee-dip-418420')  # <--- replace with your project ID

In [ ]:
import geemap

# EX1: calculation of spectral indices (band arithmetic)

## 1.1. select image

In [ ]:
# Define ROI point (EX: Cancún)
roi = ee.Geometry.Point(-86.85, 21.17)

# Import and filter imagery by location / date / cloud
image = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
            .filterBounds(roi)
            .filterDate('2020-01-01', '2024-10-01')
            .filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT', 10))
            .sort("CLOUD_COVERAGE_ASSESSMENT")
            .first()
            )

# Map Natural color image
Map = geemap.Map()
Map.addLayerControl()
Map.centerObject(roi, 10)
Map.addLayer(image, {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 2000
}, 'Natural color')

Map

In [ ]:
image

## 1.2. calculate spectral indices

### calculate NDVI (using math operators)

In [ ]:
# Extract bands needed for NDVI index
nir = image.select('B8')
red = image.select('B4')

# Calculate NDVI index using math operators
numerator = nir.subtract(red)
denominator = nir.add(red)
ndvi = numerator.divide(denominator)

# Add NDVI index on map
palette = ['red', 'white', 'green']
Map.addLayer(ndvi, {
    'min': -1,
    'max': 1,
    'palette': palette
}, 'NDVI manual')


# Add colorbar on map
vis_params = {'min': -1, 'max': 1, 'palette': palette}
cbar = Map.add_colorbar(vis_params, label="NDVI")

Map

### calculate NDVI (using adhoc method)

In [ ]:
# Calculate NDVI using normalizedDifference method
NDVI = image.normalizedDifference(['B8', 'B4'])

palette = ['red', 'white', 'green']
Map.addLayer(NDVI, {
    'min': -1,
    'max': 1,
    'palette': palette
}, 'NDVI')

Map

### calculate EVI (using math expression)

In [ ]:
# Compute the EVI using an expression
nir = image.select('B8')
red = image.select('B4')
blue = image.select('B2')

evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
    {
        'NIR': nir,
        'RED': red,
        'BLUE': blue
    })

Map2 = geemap.Map()
Map2.centerObject(image, 9)
Map2.addLayer(evi, {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'EVI')
Map2

## 1.3. thresholding image

### binary thresholding (.gt, .lt)

In [ ]:
# Implement a binary threshold
threshold = 0.5
img_thresh_bin = ndvi.gt(threshold)

# Map thresholded image (2 classes)
palette = ['white', 'green']
Map.addLayer(img_thresh_bin, {
    'min': 0,
    'max': 1,
    'palette': palette
}, 'Non-forest vs. Forest')

# Add colorbar on map
vis_params = {'min': 0, 'max': 1, 'palette': palette}

Map.centerObject(roi, 10)
Map.add_colorbar(vis_params, label="NDVI thresholded (2-classes)")

Map

### advanced thresholding (.where)

In [ ]:
threshold_1 = -0.1  # set water threshold
threshold_2 = 0.5   # set vegetation threshold

img_thresh = ee.Image(1)  # Initialize new thresholded image with all values = 1
img_thresh = img_thresh.clip(ndvi.geometry())   # Use clip to constrain size of the ndvi image
img_thresh = img_thresh.where(ndvi.lte(-0.1), 0)  # Make all NDVI values <= threshold_1 equal 0
img_thresh = img_thresh.where(ndvi.gte(threshold_2), 2) # Make all NDVI values >= threshold_2 equal 2

# Map thresholded image (3 classes)
palette = ['blue', 'white', 'green']
Map.addLayer(img_thresh, {
    'min': 0,
    'max': 2,
    'palette': palette
}, 'Water, Non-forest, Forest')

# Add colorbar on map
vis_params = {'min': 0, 'max': 1, 'palette': palette}
Map.add_colorbar(vis_params, label="NDVI thresholded (3-classes)")

Map

## 1.4. masking image

In [ ]:
# NDVI masking in GEE

mask = img_thresh_bin.eq(1)                     # Create a binary mask of non-forest
img_masked = img_thresh_bin.updateMask(mask)    # Update the img_thresh_bin mask with the non-forest mask
mask_final = img_masked.mask()                  # Updated mask

# Visualize updated mask
Map.addLayer(mask_final, {}, 'Mask')

# Visualize masked image
Map.addLayer(img_masked, {
    'min': 0,
    'max': 1,
    'palette': ['green']
    }, 'Masked Forest Layer')

Map

## 1.5. (remapping values in image)

In [ ]:
# Remap the values from the thresholded image
img_thresh_remap = img_thresh.remap([0, 1, 2],    # Existing values
                            [9, 11, 10])  # Remapped values

# Visualize the remapped thresholded image
vis_params = {'min': 9, 'max': 11, 'palette': ['blue', 'green', 'white']}
Map.addLayer(img_thresh_remap, vis_params, 'Remapped Values')

# Add colorbar on map
cbar = Map.add_colorbar(vis_params, label="mask (remapped values)")

Map


# EX2: Image fusion (remove/replace clouds)

Ref: https://courses.geemap.org/gee_intro/Image/conditional_operations/#expressions

In [ ]:
# Load a cloudy Landsat 8 image
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20130603')

# Load another image to replace cloudy pixels
replacement = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20130416')

# Compute a cloud score band
cloud = ee.Algorithms.Landsat.simpleCloudScore(image).select('cloud')

# Set cloudy pixels to the other image.
replaced = image.where(cloud.gt(10), replacement)

# Display result
Map = geemap.Map()
Map.addLayerControl()
Map.centerObject(image, 9)
Map.addLayer(
    image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'original image (cloudy)'
)
Map.addLayer(cloud, {}, 'cloud score')
Map.addLayer(
    replaced, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'fused image (clouds replaced)'
)
Map